In [43]:
from transformers import AutoModel, AutoTokenizer

In [44]:
'''
主要预训练模型:
'roberta-base'
'roberta-large'
'''
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
print(tokenizer.model_input_names)  # 动态编码
print(tokenizer.all_special_tokens)

['input_ids', 'attention_mask']
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [45]:
model = AutoModel.from_pretrained("roberta-base")
model

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [46]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [47]:
for name, para in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [50]:
text = """Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length.
To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer.
Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention.
Following prior work on long-sequence transformers, we evaluate Longformer on character-level language modeling and achieve state-of-the-art results on text8 and enwik8.
In contrast to most prior work, we also pretrain Longformer and finetune it on a variety of downstream tasks.
Our pretrained Longformer consistently outperforms RoBERTa on long document tasks and sets new state-of-the-art results on Wiki-Hop and TriviaQA."""
text = text * 2
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(encoded_input['input_ids'].shape)  # [1, 408]

{'input_ids': tensor([[    0, 19163, 22098,    12,   805,  3092,    32,  3276,     7,   609,
           251, 26929,   528,     7,    49,  1403,    12,  2611, 19774,  2513,
             6,    61, 21423, 15694,   338, 23050,    19,     5, 13931,  5933,
             4, 50118,  3972,  1100,    42, 22830,     6,    52,  6581,     5,
          2597, 22098,    19,    41,  1503,  9562,    14, 21423, 24248, 23099,
            19, 13931,  5933,     6,   442,    24,  1365,     7,   609,  2339,
             9,  1583,     9, 22121,    50,  1181,     4, 50118, 21001, 22098,
            17,    27,    29,  1503,  9562,    16,    10,  1874,    12,   179,
          5010,    13,     5,  2526,  1403,    12,  2611, 19774,     8, 15678,
            10,   400,  2931,   196,  1503,    19,    10,  3685,  7958,   720,
          1503,     4, 50118, 20839,  2052,   173,    15,   251,    12, 46665,
          7891,   268,     6,    52, 10516,  2597, 22098,    15,  2048,    12,
          4483,  2777, 19039,     8,  

In [42]:
result = model(**encoded_input)
print(result.last_hidden_state)
print(result.last_hidden_state.shape)  # [1, 408, 768](限制input_ids最大长度为514=512+2)
print(result.pooler_output)

tensor([[[-1.7965e-02,  1.0580e-01,  8.5783e-04,  ..., -6.5853e-02,
          -3.6090e-03, -2.2129e-02],
         [-2.5865e-03,  1.6560e-01, -1.0407e-01,  ..., -9.0544e-02,
          -4.3457e-02,  4.4587e-03],
         [ 1.0382e-02, -8.6941e-02, -1.9525e-01,  ..., -1.0271e+00,
           1.1050e-01,  4.4149e-01],
         ...,
         [-1.5290e-01, -3.3107e-01, -1.9680e-01,  ..., -2.4695e-01,
           3.6188e-01,  1.6483e-01],
         [-1.0169e-02,  1.0597e-01, -3.4931e-02,  ..., -1.3704e-01,
          -2.3220e-03, -7.7600e-02],
         [ 1.1030e-01,  7.8143e-02,  7.2378e-02,  ...,  9.1892e-02,
           9.6042e-02,  4.4672e-02]]], grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 408, 768])
tensor([[-1.0994e-02, -1.9034e-01, -1.7558e-01, -5.5791e-02,  1.2444e-01,
          1.9599e-01,  2.3791e-01, -1.0246e-01, -6.4833e-02, -1.4011e-01,
          2.6034e-01,  2.8523e-02, -1.2916e-01,  8.8719e-02, -1.1078e-01,
          4.4794e-01,  2.2751e-01, -4.6219e-01,  8.2148e-02, -2.2255e-